In [28]:
import subprocess
from astropy.table import Table
import os
import csv
#in the furture make packages and import common functions
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from pyraf import gwm
from astropy.io import ascii
import numpy as np
from astropy.stats import sigma_clip
from astropy.io import fits

In [29]:
from pyraf import iraf
from iraf import obsutil
from iraf import psfmeasure
from iraf import phot
from iraf import daofind
from iraf import daophot
from iraf import psf
from iraf import artdata
from iraf import mkobjects
from iraf import starlist

In [30]:
imagename = '/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits'
diffImage = '/home/mj1e16/Simages/diff_44_1.fits'
SImage= '/home/mj1e16/Simages/testsimage.fits'
ext = 44

In [31]:
outDir = '/home/mj1e16/iraf/outputs/' # for iraf output files ONLY
imageDir = '/home/mj1e16/Simages/irafImages/' # for images generated during process ONLY
simageDir = '/home/mj1e16/Simages/' # directory for simulated images
irafDir = '/home/mj1e16/iraf/' #Directory containing iraf installation

In [73]:
def mkobjectspluspsf(imagename,diffimage,diffPlusSim,ext,section,Xmax=1000,Xmin=0,Ymax=1000,Ymin=1000,thresh=50000,nsegs=5,goodpsf='',spatialDist='uniform',zeropoint=25.):
    
    daoout = outDir+'{}_{}daofindone'.format(ext,section)
    photout = outDir+'{}_{}photone'.format(ext,section)
    psfout = outDir+'{}_{}psfone'.format(ext,section)
    groupout = outDir+'{}_{}groupfileone'.format(ext,section)
    opstout = outDir+'{}_{}optsone'.format(ext,section)
    starfieldsect = outDir+'starfield_{}.dat'.format(section)
    imseg = imageDir+'imageseg{}.fits'.format(section)
    
    hdu = fits.open(imagename)
    print(imagename)
    image = hdu[ext].section[Xmin:Xmax,Ymin:Ymax]
    hdu = fits.PrimaryHDU(image)
    hdu.writeto(imseg,clobber=True)
    
    with open(starfieldsect,'w') as f:
        f.write("")
    
    daofind(imseg,output=daoout,sigma=1.0,threshold=thresh)
    phot(imseg,skyfile=daoout,coords=daoout,output=photout,interactive='no')
    psf(imseg,photfile=daoout,pstfile=photout,psfimage=psfout,opstfile=opstout,groupfile=groupout,interactive='no')
    starlist(starfieldsect,nstars=7000/nsegs,xmin=Xmin,xmax=Xmax,ymin=Ymin,ymax=Ymax,spatial=spatialDist)
    try:
        mkobjects(diffimage,output=diffPlusSim,objects=starfieldsect,gain=110,rdnoise=127,star=psfout+'.fits')
        
    except:
        try:
            #subprocess.call(['rm','/home/mj1e16/Simages/testsimage*'])
            mkobjects(diffimage,output=diffPlusSim,objects=starfieldsect,gain=110,rdnoise=127,star=goodpsf,magzero=zeropoint)
            print('Waring: Section {} with bad PSF, {} used inplace'.format(section,goodpsf))
        except:
            print('ERROR: PSF for Section {} should be made manually'.format(section))
    
    return psfout+'.fits'

In [72]:
def findPSFandMkObjects(imageName,diffImage,SImage,extension,imageshape=[1015., 1085.],pixelsize=3.98):
    
    print(imageName)
    
    os.chdir(irafDir)
    
    xlength = imageshape[1]
    ylength = imageshape[0]
    nsegs = 4
    
    xsegment = xlength/nsegs
    ysegment = ylength/nsegs
    PSF = []
    goodpsf=''
    for xsegs in range(nsegs):
        lowx = int(xsegs*xsegment)+15
        highx = int(lowx+xsegment)+15
        psf = []
        for ysegs in range(0,nsegs):
            lowy = int(ysegs*ysegment)+15
            highy = int(lowy+ysegment)+15
            
            
            # stars are being lost from images in xseg=0
            SImage= imageDir+'testsimage_{}_{}.fits'.format(xsegs,ysegs)
            
            goodpsf = mkobjectspluspsf(imageName,diffImage,SImage,extension,'{}_{}'.format(xsegs,ysegs),Xmax=highx,Xmin=lowx,Ymax=highy,Ymin=lowy,nsegs=nsegs**2,goodpsf=goodpsf)
            
            b = diffImage.find('_alt')
            if b == -1:
                diffImage = diffImage[:-5]+'_alt.fits'
            try:
                hdu = fits.open(SImage) #,ignore_missing_end=True)
                imdata = hdu[0].data
                hdu = fits.PrimaryHDU(imdata)
                hdu.writeto(diffImage,clobber=True)
                print('find me')
                print(SImage)
                print(diffImage)
            except:
                print('bad at section {} {}'.format(xsegs,ysegs))

            

            
    return 'complete!'

In [68]:
def cleanDirectories(dirlist):
    # clean iraf output and input images
    # make sure to have no needed files in these directories
    for directory in dirlist:
        dirfiles = os.listdir(directory)
        for eachFile in dirfiles:
            #print('rm',directory+eachFile)
            subprocess.call(['rm',directory+eachFile])

In [69]:
def stitchStarList(nsegs):
    with open(outDir+'starfield_0_0.dat') as f:
        lines = f.readlines()
    bigstring = ['# X_POS Y_POS MAG\n']
    bigstring.extend(lines[0:18])
    for x in range(nsegs):
        for y in range(nsegs):
            with open(outDir+'starfield_{}_{}.dat'.format(x,y)) as f:
                lines = f.readlines()
            bigstring.extend(lines[18:])


    with open('/home/mj1e16/keplerPhotometry/starlistFull.dat','w') as f:
        f.write(''.join(bigstring))


In [70]:
cleanDirectories([outDir,imageDir]) # cleans iraf output files, will remove all files from these directories

In [74]:
run = 'not complete'
for x in range(5):
    try:
        run = findPSFandMkObjects(imagename,diffImage,SImage,ext)
        if run == 'complete!':
            break
    except:
        pass

/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigma (50000.) (CR or value): 	New detection threshold: 50000. sigma 50000. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts


Image: /home/mj1e16/Simages/irafImages/imageseg0_0.fits  fwhmpsf: 2.5  ratio: 1.  theta: 0.  nsigma: 1.5

   52.01    24.52   -1.259   0.691   0.323   0.558     1
  148.02    97.33   -1.209   0.644   0.469   1.000     2
   47.79   121.86   -1.202   0.458   0.250   0.333     3
  220.81   142.56   -0.204   0.782  -0.087   0.276     4
   26.47   145.63   -1.268   0.684  -0.378   0.015 

find me
/home/mj1e16/Simages/irafImages/testsimage_0_0.fits
/home/mj1e16/Simages/diff_44_1_alt.fits
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigma (50000.) (CR or value): 	New detection threshold: 50000. sigma 50000. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts


Image: /home/mj1e16/Simages/irafImages/imageseg0_1.fits  fwhmpsf: 2.5  ratio: 1.  theta: 0.  nsigma: 1.5

  124.58    39.08   -0.886   0.918  -0.267  -0.216     1
  250.86    61.44   -0.919   0.675  -0.162  -0.034     2
   44.48    77.97   -0.237   0.929  -0.271  -0.295     3
  267.02    83.34   -1.037   0.839   0.075   0.169     4
  154.


Star 1 has been added to the PSF star list
	X:   42.78 Y:   51.15  Mag:  -0.111  Dmin: 116.7483  Dmax: 70371.19
Star 2 has been added to the PSF star list
	X:  203.80 Y:   63.44  Mag:  -0.365  Dmin: 165.6545  Dmax: 83672.05
Star 3 has been added to the PSF star list
	X:   12.99 Y:   83.34  Mag:  -1.116  Dmin: 161.6817  Dmax: 155434.1
Star 4 has been added to the PSF star list
	X:  122.13 Y:   86.21  Mag:  -0.942  Dmin: 146.6337  Dmax: 151195.6
Star 5 has been added to the PSF star list
	X:  190.29 Y:  143.90  Mag:  -1.229  Dmin: 159.8523  Dmax: 151366.9
Star 6 has been added to the PSF star list
	X:  227.91 Y:  158.74  Mag:  -1.321  Dmin: 163.2742  Dmax: 172440.2
Star 7 has been added to the PSF star list
	X:   22.50 Y:  197.97  Mag:  -1.350  Dmin: 165.3972  Dmax: 155870.6
Star 8 has been added to the PSF star list
	X:  199.55 Y:  247.78  Mag:  -0.952  Dmin: 162.9827  Dmax: 150202.9
Star 9 has been added to the PSF star list
	X:  107.89 Y:  258.54  Mag:  -0.986  Dmin: 166.7431  Dmax: 

imageseg1_0.fits     28.00   200.82  165.3706   10.656  ok
imageseg1_0.fits     33.50   254.66  168.7651   10.601  ok
imageseg1_0.fits     80.61   261.84  167.7047   11.438  ok
imageseg1_0.fits     32.23   284.19  169.6331    INDEF  err

Analytic psf function(s) (gauss): 	Analytic psf function(s): gauss
Order of variable psf (0): 	Order of variable psf: 0
Psf radius in scale units (11.): 	New psf radius: 11. scale units 11. pixels
Fitting radius in scale units (3.): 	New fitting radius: 3. scale units 3. pixels
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts


Computing PSF for image: /home/mj1e16/Simages/irafImages/imageseg1_0.fits
13 stars read from /home/mj1e16/iraf/outputs/44_1_0daofindone

Star 1 has been added to the PSF star list
	X:  184.46 Y:   21.41  Mag:  -0.227  Dmin: 163.9494  Dmax: 65339.73
Star 2 has been added to the PSF star list
	X:   20.

Killing IRAF task `mkobjects'


...

Killing IRAF task `mkobjects'
Killing IRAF task `mkobjects'


ERROR: PSF for Section 3_0 should be made manually
find me
/home/mj1e16/Simages/irafImages/testsimage_3_0.fits
/home/mj1e16/Simages/diff_44_1_alt.fits
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigma (50000.) (CR or value): 	New detection threshold: 50000. sigma 50000. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts


Image: /home/mj1e16/Simages/irafImages/imageseg3_1.fits  fwhmpsf: 2.5  ratio: 1.  theta: 0.  nsigma: 1.5

  253.14    14.26   -0.954   0.907   0.147   0.115     1
  240.05    48.74   -1.335   0.577   0.350   0.593     2
  242.01    94.56   -0.940   0.979   0.120   0.202     3
  141.10   1

11 stars read from /home/mj1e16/iraf/outputs/44_3_2daofindone

Star 1 has been added to the PSF star list
	X:   58.18 Y:   15.66  Mag:  -0.397  Dmin: 162.4726  Dmax: 93254.07
Star 2 has been added to the PSF star list
	X:   15.08 Y:   46.55  Mag:  -0.826  Dmin: 160.3534  Dmax: 135172.7
Star 3 has been added to the PSF star list
	X:  193.73 Y:   47.87  Mag:  -0.790  Dmin: 237.9352  Dmax: 181495.2
Star 4 is too near the edge of the image
Star 5 has been added to the PSF star list
	X:  213.81 Y:   70.56  Mag:  -0.814  Dmin: 176.6135  Dmax: 120536.7
Star 6 has been added to the PSF star list
	X:  261.05 Y:   95.57  Mag:  -1.039  Dmin: 167.1782  Dmax: 153203.1
Star 7 has been added to the PSF star list
	X:  201.42 Y:   98.77  Mag:  -0.344  Dmin: 182.1724  Dmax: 80131.44
Star 8 has been added to the PSF star list
	X:  156.65 Y:  181.78  Mag:  -0.903  Dmin: 175.1693  Dmax: 175686.9
Star 9 has been added to the PSF star list
	X:  230.78 Y:  189.54  Mag:  -1.134  Dmin: 167.0137  Dmax: 152667.2


Killing IRAF task `mkobjects'


Waring: Section 3_2 with bad PSF, /home/mj1e16/iraf/outputs/44_3_1psfone.fits used inplace
find me
/home/mj1e16/Simages/irafImages/testsimage_3_2.fits
/home/mj1e16/Simages/diff_44_1_alt.fits
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigma (50000.) (CR or value): 	New detection threshold: 50000. sigma 50000. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts


Image: /home/mj1e16/Simages/irafImages/imageseg3_3.fits  fwhmpsf: 2.5  ratio: 1.  theta: 0.  nsigma: 1.5

   30.33    17.91   -0.335   0.964  -0.221  -0.169     1
   51.76    27.48   -0.453   0.776  -0.146   0.188     2
  175.94    44.61   -1.217   

In [45]:
stitchStarList(4)